In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from collections import Counter
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

import os
print(os.listdir("../input"))

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

image_size = 100

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/labels.csv')
s = Counter(df['breed'])
x = s.keys()
y = s.values()
x = list(x)
y = list(y)
plt.bar(x,y)
plt.show()

In [ ]:
df['breed'].value_counts().sort_values(axis=0).plot(kind='barh', 
                                                    figsize=(10, 30), 
                                                    title='Dog breeds')

In [ ]:
labels_df = pd.get_dummies(df['breed'])
labels = labels_df.values

print(labels.shape)
labels[0]

In [ ]:
data_original = np.array([img_to_array(load_img('../input/train/%s.jpg'%img, target_size=(image_size, image_size), color_mode='grayscale')) for img in df['id'].values.tolist()]).astype(np.int16)
data_flipped = np.array(list(map(np.fliplr, data_original)))

print(data_original.shape, data_flipped.shape)

In [ ]:
plt.imshow(data_original[0].reshape(image_size, image_size), cmap='gray')
plt.show()
plt.imshow(data_flipped[0].reshape(image_size, image_size), cmap='gray')

In [ ]:
data = np.concatenate((data_original, data_flipped),axis=0)
data = np.true_divide(data, 255, dtype=np.float64)

labels = np.concatenate((labels, labels), axis=0)

print(data.shape, labels.shape)
plt.imshow(data[0].reshape(image_size, image_size), cmap='gray')
plt.show()
plt.imshow(data[10222].reshape(image_size, image_size), cmap='gray')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)
print(x_train.shape, y_test.shape)
del data
del labels
del data_original
del data_flipped


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), 
                 padding='same', 
                 input_shape=(image_size, image_size, 1), 
                 activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(120, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(), 
              metrics=['accuracy'])

In [ ]:
model_hist = model.fit(x_train, y_train,
                       epochs=5,
                       batch_size=16,
                       validation_data=(x_test, y_test))

In [ ]:
sample_sub_df = pd.read_csv('../input/sample_submission.csv')

sample_sub_data = np.array([img_to_array(load_img('../input/test/%s.jpg'%img, 
                                                  target_size=(image_size, image_size),
                                                  color_mode='grayscale'))
                            for img in sample_sub_df['id'].values.tolist()]).astype(np.int16)
sample_sub_data = np.true_divide(sample_sub_data, 255, dtype=np.float64)

In [ ]:
    prediction = model.predict(sample_sub_data, verbose=1)

In [ ]:
submission_df = pd.DataFrame(prediction)
columns_names = labels_df.columns.values
submission_df.columns = columns_names
submission_df.insert(0, 'id', sample_sub_df['id'])

submission_df.to_csv('answer.csv', index = False)

submission_df.head()